In [1]:
#dependencies
import pandas as pd
import numpy as np
import os

In [2]:
#files to be open
data_measur="hawaii_measurements.csv"
data_stat="hawaii_stations.csv"

In [3]:
#open csv files
data_measur_df=pd.read_csv(data_measur)
data_stat_df=pd.read_csv(data_stat)

In [4]:
#show the df
data_measur_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,NaN,73


In [5]:
#check for nans
data_measur_df[data_measur_df['prcp'].isnull()].tail()

,station,date,prcp,tobs
19532,USC00516128,2017-08-06,NaN,79
19537,USC00516128,2017-08-11,NaN,72
19539,USC00516128,2017-08-13,NaN,80
19544,USC00516128,2017-08-18,NaN,76
19546,USC00516128,2017-08-20,NaN,78


In [6]:
#show the second df
data_stat_df


,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


In [7]:
#fill nans with 0
data_measur_df.fillna(value=0,inplace=True)


In [8]:
#write to the new csv
data_measur_df.to_csv('clean_hawaii_measurements.csv',index=False)
clean_data_measur='clean_hawaii_measurements.csv'

In [9]:
clean_data_measur_df=pd.read_csv(clean_data_measur)
clean_data_measur_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,0.00,73


In [10]:
# Import SQL Alchemy
import sqlalchemy
from sqlalchemy import create_engine, MetaData

# Import PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float

In [11]:
engine = create_engine("sqlite:///hawaii.sqlite")

In [12]:
conn = engine.connect()

In [13]:
# Create Measurement and Station classes

class Measurement(Base):
    __tablename__ = 'measurement'
    id = Column(Integer, primary_key=True)
    station = Column(String(255))
    date = Column(String(255))
    prcp = Column(Integer)
    tobs = Column(Integer)

class Station(Base):
    __tablename__ = 'station'
    id = Column(Integer, primary_key=True)
    station = Column(String(255))
    name = Column(String(255))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation=(Integer)

In [14]:
Base.metadata.create_all(engine)

In [15]:
data_m = clean_data_measur_df.to_dict(orient='records')
data_s=data_stat_df.to_dict(orient='records')


In [16]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [17]:
table_m = sqlalchemy.Table('measurement', metadata, autoload=True)
table_s = sqlalchemy.Table('station', metadata, autoload=True)

In [18]:
conn.execute(table_m.delete())
conn.execute(table_s.delete())

In [19]:
conn.execute(table_m.insert(), data_m)
conn.execute(table_s.insert(), data_s)

In [20]:
conn.execute("select * from measurement limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0, 63),
 (3, 'USC00519397', '2010-01-03', 0, 74),
 (4, 'USC00519397', '2010-01-04', 0, 76),
 (5, 'USC00519397', '2010-01-06', 0, 73)]

In [21]:
conn.execute("select * from station limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111)]